In [1]:
# !wget https://huggingface.co/datasets/mesolitica/kesalahan-tatabahasa/resolve/main/train.jsonl -O tatabahasa1.jsonl
# !wget https://huggingface.co/datasets/mesolitica/kesalahan-tatabahasa/resolve/main/train2.jsonl -O tatabahasa2.jsonl

In [2]:
describe = [
    {'class': 0,
     'Description': 'PAD',
     'salah': '',
     'betul': ''},
    {'class': 1,
     'Description': 'kesambungan subwords',
     'salah': '',
     'betul': '',
     },
    {'class': 2,
     'Description': 'tiada kesalahan',
     'salah': '',
     'betul': '',
     },
    {'class': 3,
     'Description': 'kesalahan frasa nama, Perkara yang diterangkan mesti mendahului "penerang"',
     'salah': 'Cili sos',
     'betul': 'sos cili',
     },
    {'class': 4,
     'Description': 'kesalahan kata jamak',
     'salah': 'mereka-mereka',
     'betul': 'mereka',
     },
    {'class': 5,
     'Description': 'kesalahan kata penguat',
     'salah': 'sangat tinggi sekali',
     'betul': 'sangat tinggi',
     },
    {'class': 6,
     'Description': 'kata adjektif dan imbuhan "ter" tanpa penguat.',
     'salah': 'Sani mendapat markah yang tertinggi sekali.',
     'betul': 'Sani mendapat markah yang tertinggi.',
     },
    {'class': 7,
     'Description': 'kesalahan kata hubung',
     'salah': 'Sally sedang membaca bila saya tiba di rumahnya.',
     'betul': 'Sally sedang membaca apabila saya tiba di rumahnya.',
     },
    {'class': 8,
     'Description': 'kesalahan kata bilangan',
     'salah': 'Beribu peniaga tidak membayar cukai pendapatan.',
     'betul': 'Beribu-ribu peniaga tidak membayar cukai pendapatan',
     },
    {'class': 9,
     'Description': 'kesalahan kata sendi',
     'salah': 'Umar telah berpindah daripada sekolah ini bulan lalu.',
     'betul': 'Umar telah berpindah dari sekolah ini bulan lalu.',
     },
    {'class': 10,
     'Description': 'kesalahan penjodoh bilangan',
     'salah': 'Setiap orang pelajar',
     'betul': 'Setiap pelajar.',
     },
    {'class': 11,
     'Description': 'kesalahan kata ganti diri',
     'salah': 'Pencuri itu telah ditangkap. Beliau dibawa ke balai polis.',
     'betul': 'Pencuri itu telah ditangkap. Dia dibawa ke balai polis.',
     },
    {'class': 12,
     'Description': 'kesalahan ayat pasif',
     'salah': 'Cerpen itu telah dikarang oleh saya.',
     'betul': 'Cerpen itu telah saya karang.',
     },
    {'class': 13,
     'Description': 'kesalahan kata tanya',
     'salah': 'Kamu berasal dari manakah ?',
     'betul': 'Kamu berasal dari mana ?',
     },
    {'class': 14,
     'Description': 'kesalahan tanda baca',
     'salah': 'Kamu berasal dari manakah .',
     'betul': 'Kamu berasal dari mana ?',
     },
    {'class': 15,
     'Description': 'kesalahan kata kerja tak transitif',
     'salah': 'Dia kata kepada saya',
     'betul': 'Dia berkata kepada saya',
     },
    {'class': 16,
     'Description': 'kesalahan kata kerja transitif',
     'salah': 'Dia suka baca buku',
     'betul': 'Dia suka membaca buku',
     },
    {'class': 17,
     'Description': 'penggunaan kata yang tidak tepat',
     'salah': 'Tembuk Besar negeri Cina dibina oleh Shih Huang Ti.',
     'betul': 'Tembok Besar negeri Cina dibina oleh Shih Huang Ti',
     },
]

kesalahan = {}
for d in describe[2:]:
    kesalahan[d['class']] = d['Description']
    
len(kesalahan)

16

In [3]:
keys = set(kesalahan.keys())
choice = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G'}

In [4]:
from collections import defaultdict
from glob import glob
from tqdm import tqdm
import json
import random

import re

# from https://huggingface.co/Qwen/QwQ-32B-Preview

def fix_spacing(text):
    # Handle double quotes: strip spaces inside quotes
    quote_pattern = r'"([^"]*)"'
    def fix_quotes(match):
        content = match.group(1).strip()
        return f'"{content}"'
    text = re.sub(quote_pattern, fix_quotes, text)
    
    # Handle parentheses: strip spaces inside parentheses
    paren_pattern = r'\(([^)]*)\)'
    def fix_parens(match):
        content = match.group(1).strip()
        return f'({content})'
    text = re.sub(paren_pattern, fix_parens, text)
    
    # Remove spaces before standalone punctuation: ',', '.', etc.
    punct_pattern = r'\s+([,.!:])'
    text = re.sub(punct_pattern, r'\1', text)
    
    return text.replace(' ()', '')

In [5]:
chars = [
    ['(', ')'],
    ['<', '>'],
    ['{', '}'],
    ['[', ']'],
    ['<<', '>>'],
]

In [6]:
qa = []

for f in ['tatabahasa1.jsonl', 'tatabahasa2.jsonl']:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            
            if len(l[0]) < 5:
                continue
            if len(l[0]) != len(l[1]):
                continue
            for i in range(len(l[1])):
                if l[1][i][1] not in {3}:
                    t = l[1][i][0]
                    if l[1][i][1] == 2:
                        if t.istitle() or t.isupper():
                            continue
                        if len(t) < 3:
                            continue

                        if random.random() > 0.1:
                            continue

                    if random.random() > 0.3:
                        continue
                        
                    c = random.choice(chars)
                    cl = c[0]
                    cr = c[1]

                    k = [l[1][k][0] for k in range(i)] + [f'{cl}{l[1][i][0]}{cr}'] + [l[1][k][0] for k in range(i + 1, len(l[1]), 1)]
                    choices = random.sample(keys - {l[1][i][1]}, random.randint(2, 6)) + [l[1][i][1]]
                    random.shuffle(choices)
                    jawapan = [choice[no] for no, c in enumerate(choices) if c == l[1][i][1]]
                    choices = {choice[no]: kesalahan[c] for no, c in enumerate(choices)}
                    q = {
                        'soalan': f'Apakah kesalahan tatabahasa untuk {cl}{l[1][i][0]}{cr}',
                        'konteks': fix_spacing(' '.join(k)),
                        'choice': choices,
                        'jawapan': jawapan[0],
                        'pembetulan': l[0][i][0]
                    }
                    qa.append(q)
                
len(qa)

497050it [00:14, 33962.97it/s]
497079it [00:15, 33021.19it/s]


1464206

In [17]:
data = []

for l in qa:
    choice = []
    for k, v in l['choice'].items():
        choice.append(f'{k}. {v}')
    choice = '\n'.join(choice)
    s = f"{l['konteks']}\n{l['soalan']}\n{choice}"
    d = {
        'prompt_input': None,
        'input': s,
        'output': l['jawapan'],
    }
    data.append(d)

len(data)

1464206

In [18]:
import pandas as pd

df = pd.DataFrame(data)
df.to_parquet('prepared-kesalahan-tatabahasa-choice.parquet')

In [20]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-kesalahan-tatabahasa-choice.parquet",
    path_in_repo='data/kesalahan_tatabahasa_choice-00000-of-00001.parquet',
    repo_id="mesolitica/kesalahan-tatabahasa-instructions",
    repo_type="dataset",
)

prepared-kesalahan-tatabahasa-choice.parquet:   0%|          | 0.00/140M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/kesalahan-tatabahasa-instructions/commit/39d8365d094c6691698621e0c5be1640a377896a', commit_message='Upload data/kesalahan_tatabahasa_choice-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='39d8365d094c6691698621e0c5be1640a377896a', pr_url=None, pr_revision=None, pr_num=None)

In [21]:
data = []

for l in qa:
    s = f"{l['konteks']}\n{l['soalan']} dan berikan jawapan yang betul"
    a = l['choice'][l['jawapan']]
    if a != 'tiada kesalahan':
        a = f"{a}, jawapan yang betul adalah '{l['pembetulan']}'"
    d = {
        'prompt_input': None,
        'input': s,
        'output': a,
    }
    data.append(d)

In [24]:
df = pd.DataFrame(data)
df.to_parquet('prepared-kesalahan-tatabahasa-choice-fix.parquet')

In [26]:
api.upload_file(
    path_or_fileobj="prepared-kesalahan-tatabahasa-choice-fix.parquet",
    path_in_repo='data/kesalahan_tatabahasa_word_fix-00000-of-00001.parquet',
    repo_id="mesolitica/kesalahan-tatabahasa-instructions",
    repo_type="dataset",
)

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/kesalahan-tatabahasa-instructions/commit/89a78b26c140baf94924cead055b96203404feb1', commit_message='Upload data/kesalahan_tatabahasa_word_fix-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='89a78b26c140baf94924cead055b96203404feb1', pr_url=None, pr_revision=None, pr_num=None)